`Course Instructor`: **John Chiasson**

`Author (TA)`: **Ruthvik Vaila**

# Notes:
* In this notebook we shall load a large `NumPy` array directly into RAM to train a model.
* While the model is training keep an eye on the time taken and RAM usage of your machine.
* Tested on `Python 3.7.5` with `Tensorflow 1.15.0` and `Keras 2.2.4`. 
* Tested on `Python 2.7.17` with `Tensorflow 1.15.3` and `Keras 2.2.4`. 

# Imports

In [1]:
import sys, os
sys.version

'3.7.5 (default, Nov  7 2019, 10:50:52) \n[GCC 8.3.0]'

In [2]:
from tensorflow.compat.v1 import InteractiveSession
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"]="0" #setting it to -1 hides the GPU.
#tf.compat.v1.enable_eager_execution()
from tensorflow.python.client import device_lib
import numpy as np
import IPython
import sys, pickle, os, gzip
import h5py, time, inspect
import IPython.display as display
import keras, warnings
warnings.filterwarnings(action='once')
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
session = InteractiveSession(config=config)
# this make sure thaat if using a gpu total gpu memory is not gobbled
# up by tensorflow and allows growth as required

Using TensorFlow backend.


In [3]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5436599306767591548,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 5185287804566152456
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 16882190473695790363
 physical_device_desc: "device: XLA_GPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 6883875226
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6619321662014588384
 physical_device_desc: "device: 0, name: GeForce RTX 2080 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5"]

In [4]:
print('TF version:{}, Keras version:{}'.format(tf.__version__, keras.__version__))

TF version:1.15.0, Keras version:2.2.4


```
filename = 'data/emnist_train_x.h5'
with h5py.File(filename, 'r') as hf:
    train_x = hf['pool1_spike_features'][:]

filename = 'data/emnist_test_x.h5'
with h5py.File(filename, 'r') as hf:
    test_x = hf['pool1_spike_features'][:]

print('Train data shape:{}'.format(train_x.shape))
print('Test data shape:{}'.format(test_x.shape))

train_x = list(train_x) #convert 2D numpy array to a list of 1D numpy arrays 
test_x = list(test_x)

filename = 'data/emnist_train_y.pkl'
filehandle = open(filename, 'rb')
train_y = pickle.load(filehandle)
filehandle.close()

filename = 'data/emnist_test_y.pkl'
filehandle = open(filename, 'rb')
test_y = pickle.load(filehandle)
filehandle.close()

print('Train labels shape:{}'.format(train_y.shape))
print('Test labels shape:{}'.format(test_y.shape))

train_y = train_y.tolist() #convert 2D numpy array to a list of 1D numpy arrays 
test_y = test_y.tolist()
```

# Load the data

In [6]:
filename = 'data/mnist.pkl.gz'
filehandle = gzip.open(filename, 'rb')
train_data, val_data, test_data = pickle.load(filehandle, encoding='latin1')
#train_data, val_data, test_data = pickle.load(filehandle)
filehandle.close()
train_x, train_y = train_data
print('Train data shape:{} and labels shape:{}'.format(train_x.shape, train_y.shape))
val_x, val_y = val_data
print('Valid data shape:{} and labels shape:{}'.format(val_x.shape, val_y.shape))
## combine train and validation data, classifier_class can split it inside 
train_x = np.concatenate([train_x, val_x], axis=0)
train_y = np.concatenate([train_y, val_y], axis=0)
print('Train data shape:{}'.format(train_x.shape))
print('Train labels shape:{}'.format(train_y.shape))
test_x, test_y = test_data
print('Test data shape:{}'.format(test_x.shape))
print('Test labels shape:{}'.format(test_y.shape))


/home/visionteam/python37_env/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Train data shape:(50000, 784) and labels shape:(50000,)
Valid data shape:(10000, 784) and labels shape:(10000,)
Train data shape:(60000, 784)
Train labels shape:(60000,)
Test data shape:(10000, 784)
Test labels shape:(10000,)


# Train a small NN model using `tf.keras.model.fit` 
* A simple fully connected neural network with the structure
* 3630 -> 1500 -> 47
* `Adam optimizer` and `Cross Entropy Loss` with a learning rate ($\alpha$) set to `0.005`.

In [7]:
def smol_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(1500, input_dim=784, activation='relu'),
        tf.keras.layers.Dense(47)
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.005),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['sparse_categorical_accuracy'])
    return model

#CALLBACK TO KERAS TO SAVE BEST MODEL WEIGHTS
best_weights="fcn_weights_best.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(best_weights, monitor='sparse_categorical_accuracy', verbose=1, save_best_only=True,
                                                mode='max')

In [8]:
BATCH_SIZE = 32
model = smol_model()
model.summary()
history = model.fit(train_x,train_y, epochs=3, batch_size=BATCH_SIZE,validation_split=0.1, callbacks=[checkpoint])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


/home/visionteam/python37_env/lib/python3.7/site-packages/tensorflow_core/python/framework/tensor_util.py:521: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  tensor_proto.tensor_content = nparray.tostring()
/home/visionteam/python37_env/lib/python3.7/site-packages/tensorflow_core/python/training/tracking/data_structures.py:669: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(wrapped_dict, collections.Mapping):
/home/visionteam/python37_env/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:339: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(values, collections.Sequence):
/home/visionteam/python37_env/lib/python3.7/site-packages/tensorflow_core/python/framework/tensor

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1500)              1177500   
_________________________________________________________________
dense_1 (Dense)              (None, 47)                70547     
Total params: 1,248,047
Trainable params: 1,248,047
Non-trainable params: 0
_________________________________________________________________
Train on 54000 samples, validate on 6000 samples
Epoch 1/3
52960/54000 [============================>.] - ETA: 0s - loss: 0.2280 - sparse_categorical_accuracy: 0.9322
Epoch 00001: sparse_categorical_accuracy improved from -inf to 0.93276, saving model to fcn_weights_best.hdf5
54000/54000 [==============================] - 3s 52us/sample - loss: 0.2262 - sparse_categorical_accuracy: 0.9328 - val_loss: 0.1341 - val_sparse_categorical_accuracy: 0.9608
Epoch 2/3
52864/54000 [============================>.] 

# Test the model at the end

In [9]:
model.evaluate(np.array(test_x), np.array(test_y), batch_size=len(test_x))

10000/10000 [==============================] - 0s 2us/sample - loss: 0.1147 - sparse_categorical_accuracy: 0.9723


[0.11471599340438843, 0.9723]

# Test the model with best validation accuracy

In [10]:
model.load_weights(best_weights)
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.005),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['sparse_categorical_accuracy'])
model.evaluate(test_x, test_y, batch_size=len(test_x))

10000/10000 [==============================] - 0s 2us/sample - loss: 0.1147 - sparse_categorical_accuracy: 0.9723


[0.11471599340438843, 0.9723]

# Restart the notebook to free up the `GPU` and `RAM`.

In [10]:
IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel

{'status': 'ok', 'restart': True}

# Exercise: Log the RAM usage for the two cases.
* How the RAM usage varies when training the model using `.tfrecord` vs direct `NumPy` array.
* How does the speed vary?
* Plot various metrics like `training cost vs epochs`, `training accuracy vs epochs` etc. These metrics can be found in the dictionary `history` returned by the model.